In [1]:
import json
import random
import re
import pandas as pd
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

import matplotlib.pyplot as plt

# 讀取資料並做資料前處理

In [2]:
# 讀取資料
DATASET_DIR = 'E:/k++/json/lips.JSON'
with open(DATASET_DIR, encoding='utf8') as f:
    dataset = json.load(f)

In [3]:
# 讀取 stop words
STOP_WORDS_DIR = 'e://stopword.txt'
with open(STOP_WORDS_DIR, encoding='utf8') as f:
    stop_words = f.read().splitlines() 

In [4]:
# 讀取標題與內容
pname_list = list(map(lambda d: d['產品名稱'], dataset))
#pbrand_list = list(map(lambda d: d['品牌名稱'], dataset))
rtag_list = list(map(lambda d: d['推薦標籤'], dataset))
Indexs = list(map(lambda d: d['編號'], dataset))

# 執行 K-Means++

In [5]:
class KMeans:
    def cal_dist(self, p0, p1):
        """
        比較兩點的距離
        """
        return np.sqrt(np.sum((p0-p1)**2))
    
    def nearest_cluster_center(self, point, cluster_centers):
        """
        找到距離 point 最近的中心點
        """
        min_dist = float("inf")
        m = cluster_centers.shape[0]
        for i in range(m):
            d = self.cal_dist(point, cluster_centers[i])
            if min_dist > d:
                min_dist = d
        return min_dist 

    def get_centroids(self, datapoints, k):
        """
        K-means++ 演算法，取得初始化中心點
        """
        clusters = np.array([random.choice(datapoints)])
        dist = np.zeros(len(datapoints))
        
        for i in range(k-1):
            sum_dist = 0
            for j, point in enumerate(datapoints):
                dist[j] = self.nearest_cluster_center(point, clusters)
                sum_dist += dist[j]
            
            sum_dist *= random.random()
            for j, d in enumerate(dist):
                sum_dist = sum_dist - d
                if sum_dist <= 0:
                    clusters = np.append(clusters, [datapoints[j]], axis=0)
                    break
        
        return clusters
        
        
    def kmeans_plus_plus(self, datapoints, k=2):
        """
        K-means 演算法
        """
        # 定義資料維度
        d = datapoints.shape[1]
        # 最大的迭代次數
        Max_Iterations = 1000

        cluster = np.zeros(datapoints.shape[0])
        prev_cluster = np.ones(datapoints.shape[0])

        cluster_centers = self.get_centroids(datapoints, k)

        iteration = 0
        while np.array_equal(cluster, prev_cluster) is False or iteration > Max_Iterations:
            iteration += 1
            prev_cluster = cluster.copy()

            # 將每一個點做分群
            for idx, point in enumerate(datapoints):
                min_dist = float("inf")
                for c, cluster_center in enumerate(cluster_centers):
                    dist = self.cal_dist(point, cluster_center)
                    if dist < min_dist:
                        min_dist = dist  
                        cluster[idx] = c   # 指定該點屬於哪個分群

            # 更新分群的中心
            for k in range(len(cluster_centers)):
                new_center = np.zeros(d)
                members = 0
                for point, c in zip(datapoints, cluster):
                    if c == k:
                        new_center += point
                        members += 1
                if members > 0:
                    new_center = new_center / members
                cluster_centers[k] = new_center

        return cluster


# 使用 tf-idf 向量化

In [6]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(rtag_list)   #將rtag_list進行數值向量化
#tfidf = tfidf_vectorizer.fit_transform(pbrand_list) #將pbrand_list進行數值向量化 
tfidf = tfidf.toarray()


In [7]:
print(tfidf)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# 分群

In [8]:
k = 3
Kmeans_cluster = KMeans()
rtag_cluster_result = Kmeans_cluster.kmeans_plus_plus(tfidf, k)     #進行比對, 此用rtag,pbrand為依據
#pbrand_cluster_result = Kmeans_cluster.kmeans_plus_plus(tfidf, k)
cluster = [[] for _ in range(k)]
cluster_index = [[] for _ in range(k)]

for idx, c in enumerate(rtag_cluster_result):
    
    cluster_index[int(c)].append(Indexs[idx])
    
    cluster[int(c)].append(pname_list[idx])
    
for c, result in enumerate(cluster):
    
    print('Cluster {}:\n{}'.format(c, '\n'.join(result)))
    print("=============================================")
    print("=============================================")

Cluster 0:
LMTNARY閃亮持久唇蜜
變色粉紅彩妝潤唇膏
嘟嘟Q潤翹唇凍
淡彩潤唇蜜
木軸唇線筆
灩色唇釉
淡彩果凍潤唇蜜
水潤Q唇蜜
HONEY豐潤唇蜜<進化版>
水亮唇蜜(透明)
蜜糖沁潤唇蜜
CC唇部底膏(持色型) CC LIP PRIMER (TINT)
CC多機能潤彩唇膏 CC LIP CREAM
CC色季唯我多機能唇膏 CC PERSONAL LIP CREAM
潤彩立體唇蜜
LIP THE
惹我 唇膏
Lip Baby 漾彩潤唇蠟筆
玩美顯色唇線筆
輕輕親吻輕唇膏(濃郁顯色型)
膠原亮澤唇蜜
自轉式唇線筆
薔薇愛戀潤唇蜜 HAPPY BATH DAY Precious Rose Rose Enrich Lip Essence
魔女之唇口紅晶棒
親親彈潤豐唇蜜
變粧幻色唇蜜
潤彩唇膏
ff 琉璃光迷你口紅
糖心砰砰唇膏
唇蜜
持久潤澤唇膏
BB晶亮唇蜜 BB Lip Gloss
魔幻變色保濕唇蜜
滋潤亮唇彩 Lipsticker Water Type
閃耀唇蜜 Frostshiny Gloss
晶透水光唇凍
輕染潤唇采筆
啾啾豐唇采筆
滋潤持久型唇膏
晶透唇膏
水潤晶透唇膏
長效潤采唇膏 CELFUL ELSIA LIPSTICK N
澄透潤感唇膏
閃耀持色唇釉
超玩美艷光唇蜜
超玩美艷光唇蜜(金屬光)
幻夢持色唇蜜
超出色艷遇唇彩筆
艷澤漆光唇釉
晶俏保濕蜜唇膏
魔法胭脂唇頰蜜
魔法胭脂唇頰兩用蜜
女伶戀彩唇凍
裸唇蜜 NUDY GLOW
蜜糖水吻唇凍
著色唇釉
果漾唇釉
亮彩唇膏 Glossy Touch
蜜糖水吻唇蜜
晶采水潤唇蜜
蜜糖水吻唇蜜(霧面)
魅惑雙效唇蜜
CC多機能潤彩唇頰霜N
唇線繪畫筆 PENCIL LIP LINER
膠原豐潤唇膏
水漾粼光唇膏
流光潤彩唇膏 SMOOTH COLOR ROUGE
雙彩雙唇筆
輕感果漾唇蜜 CANDY WRAP LIP
晶耀蜜口紅
美発色唇膏
唇彩水蠟筆 Stay-On Balm Rouge
唇彩水蠟筆(持久版)
綺耀水潤唇釉 AQUARY BALM ROUGE
唇膏
天使之吻修潤口紅 FASIO BALM ROUGE
太陽之吻持色口紅
漾彩泡泡口紅蜜
鑽石巨星豐潤唇蜜
UV潤彩嘟嘟護唇蜜SPF25/PA+
粉櫻漸層唇彩筆
幻色持久唇釉N
造霧者艷色唇釉
日不落驚艷保濕唇膏
水漾潤澤唇膏

In [9]:
print(type(cluster[0][0]))
print(type(cluster_index[0][0]))

<class 'str'>
<class 'int'>


In [10]:
print(type('Cluster {}: {}'.format(c, ' '.join(result))))

<class 'str'>


In [11]:
cluster_index

[[3,
  8,
  24,
  25,
  26,
  27,
  28,
  40,
  41,
  43,
  48,
  56,
  59,
  60,
  62,
  63,
  66,
  71,
  91,
  97,
  99,
  100,
  103,
  106,
  107,
  108,
  119,
  120,
  123,
  124,
  134,
  141,
  142,
  143,
  151,
  152,
  156,
  157,
  169,
  170,
  171,
  172,
  173,
  174,
  181,
  183,
  184,
  189,
  193,
  206,
  207,
  208,
  209,
  210,
  213,
  216,
  217,
  220,
  222,
  223,
  224,
  225,
  235,
  237,
  241,
  242,
  244,
  252,
  254,
  255,
  256,
  262,
  263,
  267,
  269,
  273,
  275,
  284,
  285,
  289,
  294,
  295,
  297,
  303,
  308,
  309,
  332,
  333,
  335,
  338,
  339,
  340,
  341,
  343,
  346,
  350,
  351,
  363,
  365,
  370,
  371,
  376,
  385,
  391,
  396,
  403,
  404,
  406,
  409,
  415,
  416,
  417,
  438,
  439,
  473,
  482,
  486,
  488,
  499,
  512,
  514,
  518,
  520,
  521,
  522,
  526,
  530,
  533,
  535,
  555,
  558,
  563,
  576,
  600,
  602,
  604,
  615,
  622,
  623,
  634,
  637,
  638,
  640,
  642,
  645,
  659,
 

In [12]:
dataset[0]

{'編號': 0,
 '部位': '唇部',
 '產品分類': '唇彩',
 '產品線': '唇膏',
 '通路名稱': '開架',
 '產品名稱': 'Mineral Mineral Lipstick',
 '品牌名稱': 'e.l.f.',
 '容量': '遺漏',
 '價格': 5,
 '價格分類': 1,
 '推薦標籤': '海外販售',
 '推薦指數': 2.2,
 '人氣': 170,
 '圖片網址': '無商品圖片',
 '產品頁': 'https://www.urcosme.com/products/38680',
 '購買頁': 'https://www.urcosme.com/products/38680/sell-channels',
 '文章頁': 'https://www.urcosme.com//reviews/764402'}

In [13]:
########## 我們要得結果 ##########

for c in range(3):
    for i in cluster_index[c]:
        print("cluster %s : %s" %(c ,dataset[i]))
    

cluster 0 : {'編號': 3, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇蜜', '通路名稱': '專賣店', '產品名稱': 'LMTNARY閃亮持久唇蜜', '品牌名稱': 'DAISO 大創', '容量': '4.5g', '價格': 39, '價格分類': 1, '推薦標籤': ' 日本 J-Beauty 日本彩妝', '推薦指數': 3.4, '人氣': 762, '圖片網址': '無商品圖片', '產品頁': 'https://www.urcosme.com/products/37192', '購買頁': 'https://www.urcosme.com/products/37192/sell-channels', '文章頁': 'https://www.urcosme.com//reviews/670799'}
cluster 0 : {'編號': 8, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇膏', '通路名稱': '開架', '產品名稱': '變色粉紅彩妝潤唇膏', '品牌名稱': 'OMI 近江兄弟社', '容量': '3.7g', '價格': 90, '價格分類': 1, '推薦標籤': '顯色度佳  日本 滋潤 保濕 自然  顯色  J-Beauty 日本彩妝', '推薦指數': 5.3, '人氣': 24979, '圖片網址': 'https://dg9ugnb21lig7.cloudfront.net/uploads/review_image/1137442/_650x0_20171005092040.PNG', '產品頁': 'https://www.urcosme.com/products/31580', '購買頁': 'https://www.urcosme.com/products/31580/sell-channels', '文章頁': 'https://www.urcosme.com//reviews/924036'}
cluster 0 : {'編號': 24, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇蜜', '通路名稱': '網路', '產品名稱': '嘟嘟Q潤翹唇凍', '品牌名稱': 'HANAKA 花戀肌', '容量'

In [14]:
y = []
for c in range(3):
    for i in cluster_index[c]:
        
        x = ("cluster %s : %s" %(c ,dataset[i]))
        y.append(x)
print(y)

["cluster 0 : {'編號': 3, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇蜜', '通路名稱': '專賣店', '產品名稱': 'LMTNARY閃亮持久唇蜜', '品牌名稱': 'DAISO 大創', '容量': '4.5g', '價格': 39, '價格分類': 1, '推薦標籤': ' 日本 J-Beauty 日本彩妝', '推薦指數': 3.4, '人氣': 762, '圖片網址': '無商品圖片', '產品頁': 'https://www.urcosme.com/products/37192', '購買頁': 'https://www.urcosme.com/products/37192/sell-channels', '文章頁': 'https://www.urcosme.com//reviews/670799'}", "cluster 0 : {'編號': 8, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇膏', '通路名稱': '開架', '產品名稱': '變色粉紅彩妝潤唇膏', '品牌名稱': 'OMI 近江兄弟社', '容量': '3.7g', '價格': 90, '價格分類': 1, '推薦標籤': '顯色度佳  日本 滋潤 保濕 自然  顯色  J-Beauty 日本彩妝', '推薦指數': 5.3, '人氣': 24979, '圖片網址': 'https://dg9ugnb21lig7.cloudfront.net/uploads/review_image/1137442/_650x0_20171005092040.PNG', '產品頁': 'https://www.urcosme.com/products/31580', '購買頁': 'https://www.urcosme.com/products/31580/sell-channels', '文章頁': 'https://www.urcosme.com//reviews/924036'}", "cluster 0 : {'編號': 24, '部位': '唇部', '產品分類': '唇彩', '產品線': '唇蜜', '通路名稱': '網路', '產品名稱': '嘟嘟Q潤翹唇凍', '品牌名稱': 'HANAKA 花戀

In [15]:
###### .join( ) 裡面只能放 list 型態 ######
for c, list_content in enumerate(cluster):
    print("第 %s 個 list : " %(c) ) 
    print('Cluster {}:\n{}'.format(c, '\n'.join(list_content)))

第 0 個 list : 
Cluster 0:
LMTNARY閃亮持久唇蜜
變色粉紅彩妝潤唇膏
嘟嘟Q潤翹唇凍
淡彩潤唇蜜
木軸唇線筆
灩色唇釉
淡彩果凍潤唇蜜
水潤Q唇蜜
HONEY豐潤唇蜜<進化版>
水亮唇蜜(透明)
蜜糖沁潤唇蜜
CC唇部底膏(持色型) CC LIP PRIMER (TINT)
CC多機能潤彩唇膏 CC LIP CREAM
CC色季唯我多機能唇膏 CC PERSONAL LIP CREAM
潤彩立體唇蜜
LIP THE
惹我 唇膏
Lip Baby 漾彩潤唇蠟筆
玩美顯色唇線筆
輕輕親吻輕唇膏(濃郁顯色型)
膠原亮澤唇蜜
自轉式唇線筆
薔薇愛戀潤唇蜜 HAPPY BATH DAY Precious Rose Rose Enrich Lip Essence
魔女之唇口紅晶棒
親親彈潤豐唇蜜
變粧幻色唇蜜
潤彩唇膏
ff 琉璃光迷你口紅
糖心砰砰唇膏
唇蜜
持久潤澤唇膏
BB晶亮唇蜜 BB Lip Gloss
魔幻變色保濕唇蜜
滋潤亮唇彩 Lipsticker Water Type
閃耀唇蜜 Frostshiny Gloss
晶透水光唇凍
輕染潤唇采筆
啾啾豐唇采筆
滋潤持久型唇膏
晶透唇膏
水潤晶透唇膏
長效潤采唇膏 CELFUL ELSIA LIPSTICK N
澄透潤感唇膏
閃耀持色唇釉
超玩美艷光唇蜜
超玩美艷光唇蜜(金屬光)
幻夢持色唇蜜
超出色艷遇唇彩筆
艷澤漆光唇釉
晶俏保濕蜜唇膏
魔法胭脂唇頰蜜
魔法胭脂唇頰兩用蜜
女伶戀彩唇凍
裸唇蜜 NUDY GLOW
蜜糖水吻唇凍
著色唇釉
果漾唇釉
亮彩唇膏 Glossy Touch
蜜糖水吻唇蜜
晶采水潤唇蜜
蜜糖水吻唇蜜(霧面)
魅惑雙效唇蜜
CC多機能潤彩唇頰霜N
唇線繪畫筆 PENCIL LIP LINER
膠原豐潤唇膏
水漾粼光唇膏
流光潤彩唇膏 SMOOTH COLOR ROUGE
雙彩雙唇筆
輕感果漾唇蜜 CANDY WRAP LIP
晶耀蜜口紅
美発色唇膏
唇彩水蠟筆 Stay-On Balm Rouge
唇彩水蠟筆(持久版)
綺耀水潤唇釉 AQUARY BALM ROUGE
唇膏
天使之吻修潤口紅 FASIO BALM ROUGE
太陽之吻持色口紅
漾彩泡泡口紅蜜
鑽石巨星豐潤唇蜜
UV潤彩嘟嘟護唇蜜SPF25/PA+
粉櫻漸層唇彩筆
幻色持久唇釉N
造霧者艷色唇釉
日不

In [16]:
dataset[36]

{'編號': 36,
 '部位': '唇部',
 '產品分類': '唇彩',
 '產品線': '唇釉',
 '通路名稱': '專賣店',
 '產品名稱': '口卡啦POP普普風俏漾唇彩(迷你Q版)',
 '品牌名稱': 'ETUDE HOUSE',
 '容量': '2g',
 '價格': 150,
 '價格分類': 1,
 '推薦標籤': ' 韓國  滋潤 保濕 J-Beauty 韓國彩妝 韓國彩妝',
 '推薦指數': 3,
 '人氣': 139,
 '圖片網址': 'https://dg9ugnb21lig7.cloudfront.net/uploads/product_image/62570/1/_250x250_62570_w_250xh_250xfar_C.jpg',
 '產品頁': 'https://www.urcosme.com/products/62570',
 '購買頁': 'https://www.urcosme.com/products/62570/sell-channels',
 '文章頁': 'https://www.urcosme.com//reviews/985793'}

In [17]:
for number in cluster_index[0]:
    print(dataset[number]["產品名稱"])

LMTNARY閃亮持久唇蜜
變色粉紅彩妝潤唇膏
嘟嘟Q潤翹唇凍
淡彩潤唇蜜
木軸唇線筆
灩色唇釉
淡彩果凍潤唇蜜
水潤Q唇蜜
HONEY豐潤唇蜜<進化版>
水亮唇蜜(透明)
蜜糖沁潤唇蜜
CC唇部底膏(持色型) CC LIP PRIMER (TINT)
CC多機能潤彩唇膏 CC LIP CREAM
CC色季唯我多機能唇膏 CC PERSONAL LIP CREAM
潤彩立體唇蜜
LIP THE
惹我 唇膏
Lip Baby 漾彩潤唇蠟筆
玩美顯色唇線筆
輕輕親吻輕唇膏(濃郁顯色型)
膠原亮澤唇蜜
自轉式唇線筆
薔薇愛戀潤唇蜜 HAPPY BATH DAY Precious Rose Rose Enrich Lip Essence
魔女之唇口紅晶棒
親親彈潤豐唇蜜
變粧幻色唇蜜
潤彩唇膏
ff 琉璃光迷你口紅
糖心砰砰唇膏
唇蜜
持久潤澤唇膏
BB晶亮唇蜜 BB Lip Gloss
魔幻變色保濕唇蜜
滋潤亮唇彩 Lipsticker Water Type
閃耀唇蜜 Frostshiny Gloss
晶透水光唇凍
輕染潤唇采筆
啾啾豐唇采筆
滋潤持久型唇膏
晶透唇膏
水潤晶透唇膏
長效潤采唇膏 CELFUL ELSIA LIPSTICK N
澄透潤感唇膏
閃耀持色唇釉
超玩美艷光唇蜜
超玩美艷光唇蜜(金屬光)
幻夢持色唇蜜
超出色艷遇唇彩筆
艷澤漆光唇釉
晶俏保濕蜜唇膏
魔法胭脂唇頰蜜
魔法胭脂唇頰兩用蜜
女伶戀彩唇凍
裸唇蜜 NUDY GLOW
蜜糖水吻唇凍
著色唇釉
果漾唇釉
亮彩唇膏 Glossy Touch
蜜糖水吻唇蜜
晶采水潤唇蜜
蜜糖水吻唇蜜(霧面)
魅惑雙效唇蜜
CC多機能潤彩唇頰霜N
唇線繪畫筆 PENCIL LIP LINER
膠原豐潤唇膏
水漾粼光唇膏
流光潤彩唇膏 SMOOTH COLOR ROUGE
雙彩雙唇筆
輕感果漾唇蜜 CANDY WRAP LIP
晶耀蜜口紅
美発色唇膏
唇彩水蠟筆 Stay-On Balm Rouge
唇彩水蠟筆(持久版)
綺耀水潤唇釉 AQUARY BALM ROUGE
唇膏
天使之吻修潤口紅 FASIO BALM ROUGE
太陽之吻持色口紅
漾彩泡泡口紅蜜
鑽石巨星豐潤唇蜜
UV潤彩嘟嘟護唇蜜SPF25/PA+
粉櫻漸層唇彩筆
幻色持久唇釉N
造霧者艷色唇釉
日不落驚艷保濕唇膏
水漾潤澤唇膏
輕吻戀唇蜜
零補粧輕